### **Documentação:** Calculadora de Quality Score (AurumQualityScoreCalculator)

#### **1. Objetivo**

Este script é um dos pilares centrais do **Pilar 1 (Qualidade Financeira)** do projeto Aurum. Sua responsabilidade é transformar os dados fundamentalistas brutos (extraídos da CVM e formatados no `fundamentals_wide.parquet`) em métricas de performance acionáveis.

O script executa um pipeline completo que:
1.  Carrega os dados brutos da CVM.
2.  Calcula os valores **TTM (Trailing Twelve Months / Últimos 12 Meses)**, corrigindo a natureza "Acumulada no Ano" (YTD) dos dados da CVM.
3.  Calcula um conjunto abrangente de **ratios financeiros** (Rentabilidade, Margens, Alavancagem, etc.) usando os dados TTM.
4.  Calcula um **Aurum Quality Score** inicial, baseado em um *ranking percentile cross-sectional* (por data) desses ratios.
5.  Salva os resultados históricos e os mais recentes.

#### **2. Configuração (Input)**

O script depende de um único arquivo de entrada, que deve ser gerado pelo pipeline de processamento da CVM:

* **`data/cvm/final/fundamentals_wide.parquet**: Um arquivo Parquet contendo os dados de Balanço Patrimonial (BP) e Demonstração do Resultado (DRE) em formato "wide" (uma linha por empresa/data, múltiplas colunas de métricas).
    * **Importante:** O script assume que os dados da DRE (ex: `Receita Líquida`, `Lucro Bruto`) estão no formato **YTD (Acumulado no Ano)**, que é o padrão da CVM.

#### 3. Pipeline de Execução (Passo a Passo)

A classe **AurumQualityScoreCalculator** gerencia todo o fluxo de trabalho:

##### **Passo 1:** Carregar e Preparar Dados (`load_and_prepare_data`)

* Carrega o arquivo `fundamentals_wide.parquet`.
* Chama _clean_fundamentals_data para:
    * Converter DT_FIM_EXERC para datetime.
    * Remover quaisquer linhas duplicadas por CNPJ_CIA e DT_FIM_EXERC.
    * **Ordenar** os dados por CNPJ_CIA e DT_FIM_EXERC, o que é **essencial** para o cálculo de TTM.

##### **Passo 2:** Cálculo de TTM (`calculate_ttm_data`)

Este é o passo mais crítico do script. Ele converte os dados de fluxo (DRE) de YTD para TTM.

* **Lógica:** O script agrupa por `CNPJ_CIA` e aplica a função `rolling_sum_group`.
* **Desacumulação (YTD -> Trimestral):** Para cada coluna de DRE, ele calcula o valor trimestral fazendo:
    quarterly = group[col] - group[col].shift(1).fillna(0)
* **Anualização (Trimestral -> TTM):** Em seguida, ele calcula a soma móvel dos últimos 4 valores trimestrais:
    group[f'{col}_ttm'] = quarterly.rolling(window=4, min_periods=4).sum()
* O resultado é salvo em self.ratios_df, pronto para o cálculo dos ratios.

##### Passo 3: Cálculo de Ratios Financeiros (`calculate_financial_ratios`)

* Usando o self.ratios_df (que agora contém as colunas `_ttm`), este método calcula os principais indicadores financeiros.
* **Categorias:**
    * **Contas de Dívida: Dívida Bruta, Dívida Líquida, Capital Investido**.
    * **Rentabilidade (com TTM):** ROE, ROA, ROIC.
    * **Margens (com TTM):** MARGEM_EBIT, MARGEM_LIQUIDA, MARGEM_BRUTA.
    * **Alavancagem:** ALAVANCAGEM (Passivo/Ativo), DIVIDA_PL, DIVIDA_LIQ_EBIT.
    * **Liquidez:** LIQUIDEZ_CORRENTE.
    * **Eficiência (com TTM):** GIRO_ATIVO.
* Usa safe_divide para evitar erros de divisão por zero.

##### Passo 3.1: Tratamento de Outliers (`_handle_ratio_outliers`)

* Após o cálculo, os ratios são limpos.
* Valores `infinitos` são substituídos por `NaN`.
* Os dados são "winsorizados": valores extremos são "clipados" (limitados) aos **percentis 1% (inferior) e 99% (superior)**. Isso torna o scoring subsequente mais robusto.

##### Passo 4: Cálculo do Score de Qualidade (`calculate_quality_scores`)

Este método implementa a **Metodologia de Ranking Cross-Sectional**.

* **Configuração:** Define as métricas, pesos e a direção (ex: `ROIC` -> maior é melhor, `DIVIDA_LIQ_EBIT` -> menor é melhor).
* **Ranking por Data:** O script agrupa os dados por `DT_FIM_EXERC` (`grouped_by_date`).
* Para cada métrica, ele calcula o **ranking percentile** de cada empresa *dentro daquele período*:
    `scores_df[score_col] = grouped_by_date[metric].rank(pct=True) * 100`
* **Score Composto:** O `aurum_quality_score` é calculado como a soma ponderada desses rankings (percentis de 0 a 100). Valores `NaN` são preenchidos com a mediana (50) para não penalizar empresas excessivamente.
* **Classificação Final:** O script calcula `quality_quintile` e `quality_grade` (A-E) com base no ranking *final* do `aurum_quality_score` de cada período.

##### Passo 5 e 6: Salvar Resultados (`get_latest_scores` e `save_results`)

* O script gera e salva quatro arquivos distintos no diretório `data/aurum_scores_output/`.

#### 6. Fluxograma
```mermaid
flowchart TD
    %% --- INPUT ---
    Input[("📂 <b>Input</b><br/>fundamentals_wide.parquet")]
    
    %% --- PREPARAÇÃO ---
    subgraph Prep ["1. Preparação"]
        Clean["<b>Limpeza & Ordenação</b><br/>Sort by: CNPJ + Data"]
    end
    
    %% --- TTM ---
    subgraph TTM ["2. Cálculo TTM"]
        Desac["<b>Desacumulação</b><br/>YTD → Trimestral"]
        Roll["<b>Soma Móvel (Rolling 4)</b><br/>Trimestral → TTM (12 Meses)"]
    end
    
    %% --- RATIOS ---
    subgraph Ratios ["3. Ratios Financeiros"]
        CalcRat["<b>Cálculo de Indicadores</b><br/>ROIC, ROE, Dívida Líquida..."]
        Outliers["<b>Tratamento de Outliers</b><br/>Winsorization (Clip 1% - 99%)"]
    end
    
    %% --- SCORING ---
    subgraph Score ["4. Scoring Cross-Sectional"]
        Group["<b>Agrupamento por Data</b><br/>(Comparação Temporal Justa)"]
        Rank["<b>Ranking Percentile</b><br/>0 a 100 para cada métrica"]
        Weight["<b>Média Ponderada</b><br/>Aplicação dos Pesos (ex: ROIC 25%)"]
        Grade["<b>Classificação Final</b><br/>Quintis e Notas (A-E)"]
    end
    
    %% --- OUTPUTS ---
    subgraph Outputs ["5. Saída"]
        Hist[("💾 <b>Histórico Completo</b><br/>.parquet / .csv")]
        Latest[("💾 <b>Último Período</b><br/>.parquet / .csv")]
        Stats[("📄 <b>Relatório Estatístico</b><br/>.txt")]
    end

    %% --- LIGAÇÕES ---
    Input --> Clean
    Clean --> Desac --> Roll
    Roll --> CalcRat --> Outliers
    Outliers --> Group --> Rank --> Weight --> Grade
    Grade --> Hist & Latest & Stats
```
#### 7. Saída (Output)

A execução do script gera os seguintes arquivos em `data/aurum_scores_output/`:

1.  **`aurum_quality_scores_complete.parquet`**:
    * O arquivo mais importante. Contém o **histórico completo** de todas as empresas, datas, ratios calculados e scores. Este será o input para o `AurumScoringSystem` avançado e para o *backtesting*.

2.  **`aurum_quality_scores_latest.parquet`**:
    * Um arquivo de conveniência que contém apenas o **último registro (data mais recente)** de score/ratios para cada empresa.

3.  **`aurum_quality_scores_latest.csv`**:
    * Versão CSV do arquivo acima, para fácil visualização em planilhas.

4.  **`aurum_scores_statistics.txt`**:
    * Um relatório de texto (`.txt`) legível, mostrando as estatísticas do último período (distribuição de notas, Top 10 empresas) para uma verificação rápida.

---

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
tqdm.pandas()

class AurumQualityScoreCalculator:
    """ Calcula TTM, ratios financeiros (incluindo ROIC correto) e scores """

    def __init__(self, fundamentals_path: str):
        self.fundamentals_path = Path(fundamentals_path)
        self.fundamentals_df = None
        self.ratios_df = None
        self.scores_df = None
        self.dre_cols = [
            'Receita Líquida', 'Custo dos Bens e/ou Serviços Vendidos',
            'Lucro Bruto', 'EBIT', 'EBT', 'Lucro Líquido Consolidado'
        ]

    def load_and_prepare_data(self) -> pd.DataFrame:
        logger.info("📥 1. CARREGANDO NOVO fundamentals_wide.parquet...")
        try:
            self.fundamentals_df = pd.read_parquet(self.fundamentals_path)
            logger.info(f"✅ Dados carregados: {self.fundamentals_df.shape}")
            logger.info(f"Colunas encontradas: {self.fundamentals_df.columns.tolist()}")
            new_cols = ['Caixa e Equivalentes', 'Dívida Curto Prazo', 'Dívida Longo Prazo']
            missing_new = [col for col in new_cols if col not in self.fundamentals_df.columns]
            if missing_new:
                 logger.error(f"❌ ERRO: Novas colunas {missing_new} NÃO encontradas no input!")
                 raise ValueError(f"Novas colunas faltando: {missing_new}")
            else:
                 logger.info("✅ Novas colunas (Caixa, Dívida CP, Dívida LP) encontradas!")

            self.fundamentals_df = self._clean_fundamentals_data(self.fundamentals_df)
            return self.fundamentals_df
        except Exception as e:
            logger.error(f"❌ Erro ao carregar dados: {e}")
            raise

    def _clean_fundamentals_data(self, df: pd.DataFrame) -> pd.DataFrame:
        df_clean = df.copy()
        df_clean['DT_FIM_EXERC'] = pd.to_datetime(df_clean['DT_FIM_EXERC'], errors='coerce')
        df_clean = df_clean.dropna(subset=['DT_FIM_EXERC'])
        df_clean = df_clean.drop_duplicates(subset=['CNPJ_CIA', 'DT_FIM_EXERC'])
        df_clean = df_clean.sort_values(['CNPJ_CIA', 'DT_FIM_EXERC'])
        return df_clean

    def calculate_ttm_data(self) -> pd.DataFrame:
        if self.fundamentals_df is None: raise ValueError("Dados não carregados.")
        logger.info("⏳ 2. CALCULANDO TTM...")
        df_ttm = self.fundamentals_df.copy()
        def rolling_sum_group(group):
            for col in self.dre_cols:
                if col in group.columns:
                    quarterly = group[col] - group[col].shift(1).fillna(0)
                    group[f'{col}_ttm'] = quarterly.rolling(window=4, min_periods=4).sum()
                else: group[f'{col}_ttm'] = np.nan
            return group
        grouped = df_ttm.groupby('CNPJ_CIA', group_keys=False)
        self.ratios_df = grouped.progress_apply(rolling_sum_group)
        logger.info(f"✅ TTM calculado.")
        return self.ratios_df

    def calculate_financial_ratios(self) -> pd.DataFrame:
        """ PASSO 3: Calcula ratios (com ROIC correto e ratios de dívida) """
        if self.ratios_df is None: raise ValueError("Dados TTM não calculados.")
        logger.info("🧮 3. CALCULANDO RATIOS FINANCEIROS (COM TTM)...")
        df = self.ratios_df.copy()

        def get_col(df, col_name): return df.get(col_name, np.nan)
        def safe_divide(num, den): return np.where(den == 0, np.nan, num / den)

        divida_cp = get_col(df, 'Dívida Curto Prazo').fillna(0) 
        divida_lp = get_col(df, 'Dívida Longo Prazo').fillna(0)
        caixa = get_col(df, 'Caixa e Equivalentes').fillna(0)   
        pl = get_col(df, 'Patrimônio Líquido Consolidado').fillna(0)
        
        df['Dívida Bruta'] = divida_cp + divida_lp
        df['Capital Investido'] = df['Dívida Bruta'] + pl 
        df['Dívida Líquida'] = df['Dívida Bruta'] - caixa 
        logger.info("✅ Dívida Bruta, Capital Investido e Dívida Líquida calculados.")

        logger.info("📈 Calculando RENTABILIDADE...")
        df['ROE'] = safe_divide(get_col(df, 'Lucro Líquido Consolidado_ttm'), pl)
        df['ROA'] = safe_divide(get_col(df, 'Lucro Líquido Consolidado_ttm'), get_col(df, 'Ativo Total'))
        df['ROIC'] = safe_divide(get_col(df, 'EBIT_ttm'), df['Capital Investido'])
        logger.info(f"✅ ROIC (correto) calculado. Média: {df['ROIC'].mean():.4f}")

        df['MARGEM_EBIT'] = safe_divide(get_col(df, 'EBIT_ttm'), get_col(df, 'Receita Líquida_ttm'))
        df['MARGEM_LIQUIDA'] = safe_divide(get_col(df, 'Lucro Líquido Consolidado_ttm'), get_col(df, 'Receita Líquida_ttm'))
        df['MARGEM_BRUTA'] = safe_divide(get_col(df, 'Lucro Bruto_ttm'), get_col(df, 'Receita Líquida_ttm'))

        logger.info("⚖️ Calculando ALAVANCAGEM...")
        df['ALAVANCAGEM'] = safe_divide(get_col(df, 'Passivo Total'), get_col(df, 'Ativo Total'))
        df['DIVIDA_PL'] = safe_divide(df['Dívida Bruta'], pl) 
        df['DIVIDA_LIQ_EBIT'] = safe_divide(df['Dívida Líquida'], get_col(df, 'EBIT_ttm')) 
        logger.info("✅ Ratios de alavancagem (incluindo DIVIDA_PL, DIVIDA_LIQ_EBIT) calculados.")

        df['LIQUIDEZ_CORRENTE'] = safe_divide(get_col(df, 'Ativo Circulante'), get_col(df, 'Passivo Circulante'))

        df['GIRO_ATIVO'] = safe_divide(get_col(df, 'Receita Líquida_ttm'), get_col(df, 'Ativo Total'))

        self.ratios_df = self._handle_ratio_outliers(df)
        logger.info("🎯 RATIOS CALCULADOS e limpos.")
        logger.info(f"Colunas FINAIS de Ratios: {self.ratios_df.columns.tolist()}")
        return self.ratios_df

    def _handle_ratio_outliers(self, df: pd.DataFrame) -> pd.DataFrame:
        df_clean = df.copy()
        ratio_columns = [
            'ROE', 'ROA', 'ROIC', 'MARGEM_EBIT', 'MARGEM_LIQUIDA', 'MARGEM_BRUTA',
            'ALAVANCAGEM', 'DIVIDA_PL', 'DIVIDA_LIQ_EBIT', 'LIQUIDEZ_CORRENTE', 'GIRO_ATIVO'
        ]
        for col in ratio_columns:
            if col in df_clean.columns:
                df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
                if df_clean[col].notna().sum() > 0:
                    lower = df_clean[col].quantile(0.01)
                    upper = df_clean[col].quantile(0.99)
                    df_clean[col] = df_clean[col].clip(lower=lower, upper=upper)
        return df_clean

    def calculate_quality_scores(self) -> pd.DataFrame:
        """ PASSO 4: Calcula scores individuais e score composto """
        if self.ratios_df is None: raise ValueError("Ratios não calculados.")
        logger.info("🎯 4. CALCULANDO SCORES DE QUALIDADE...")
        scores_df = self.ratios_df.copy()
        grouped_by_date = scores_df.groupby('DT_FIM_EXERC')

        metrics_config = {
            'ROIC': {'direction': 1, 'weight': 0.25}, 
            'ROE': {'direction': 1, 'weight': 0.15},
            'MARGEM_EBIT': {'direction': 1, 'weight': 0.15},
            'MARGEM_LIQUIDA': {'direction': 1, 'weight': 0.10},
            'DIVIDA_LIQ_EBIT': {'direction': -1, 'weight': 0.15}, 
            'LIQUIDEZ_CORRENTE': {'direction': 1, 'weight': 0.10},
            'GIRO_ATIVO': {'direction': 1, 'weight': 0.10},
            # 'ALAVANCAGEM': {'direction': -1, 'weight': 0.0}, # Pode remover ou ajustar peso
        }
        total_w = sum(c['weight'] for c in metrics_config.values())
        if abs(total_w - 1.0) > 0.01:
             logger.warning(f"Soma dos pesos é {total_w:.2f}. Ajustando proporcionalmente...")
             for k in metrics_config: metrics_config[k]['weight'] /= total_w

        logger.info(f"Configuração de métricas para score: { {k: v['weight'] for k, v in metrics_config.items()} }")

        for metric, config in metrics_config.items():
            if metric in scores_df.columns and scores_df[metric].notna().any():
                score_col = f'score_{metric}'
                if config['direction'] == 1:
                    scores_df[score_col] = grouped_by_date[metric].rank(pct=True) * 100
                else:
                    scores_df[score_col] = grouped_by_date[metric].rank(ascending=False, pct=True) * 100
                logger.info(f"  ✅ Score {metric} calculado.")
            else:
                 logger.warning(f"Métrica '{metric}' não encontrada ou sem dados para score.")

        logger.info("⚖️ Calculando score composto...")
        scores_df['aurum_quality_score'] = 0.0
        total_applied_weight = 0.0
        for metric, config in metrics_config.items():
            score_col = f'score_{metric}'
            if score_col in scores_df.columns:
                scores_df['aurum_quality_score'] += scores_df[score_col].fillna(50) * config['weight']
                total_applied_weight += config['weight']

        if total_applied_weight > 0:
            scores_df['aurum_quality_score'] /= total_applied_weight

        logger.info("🏆 Classificando empresas...")
        try:
             valid_scores = scores_df['aurum_quality_score'].dropna()
             if not valid_scores.empty:
                  ranks = valid_scores.rank(ascending=False, pct=True)
                  scores_df['final_rank'] = ranks
                  try:
                       scores_df['quality_quintile'] = pd.qcut(scores_df['final_rank'].dropna(), 5, labels=[f'{i}º Quintil' for i in range(1, 6)])
                  except ValueError: scores_df['quality_quintile'] = pd.cut(scores_df['final_rank'].dropna(), 5, labels=False)
                  scores_df['quality_grade'] = pd.cut(scores_df['final_rank'].dropna(), bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0], labels=['A', 'B', 'C', 'D', 'E'], right=True, include_lowest=True)
             else: scores_df[['final_rank', 'quality_quintile', 'quality_grade']] = np.nan
        except Exception as e:
             logger.error(f"Erro ao calcular ranks/grades: {e}")
             scores_df[['final_rank', 'quality_quintile', 'quality_grade']] = np.nan
        
        self.scores_df = scores_df
        logger.info(f"Colunas FINAIS ANTES de salvar: {self.scores_df.columns.tolist()}")
        return scores_df

    def get_latest_scores(self) -> pd.DataFrame:
        if self.scores_df is None: raise ValueError("Scores não calculados.")
        latest_scores = self.scores_df.sort_values('DT_FIM_EXERC').groupby('CNPJ_CIA').last().reset_index()
        return latest_scores

    def save_results(self, output_dir: str = "../data/aurum_scores"):
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        if self.scores_df is not None:
            logger.info(f"Salvando DataFrame com colunas: {self.scores_df.columns.tolist()}")
            
            if 'ROIC' not in self.scores_df.columns: 
                logger.error("ERRO FATAL: Coluna 'ROIC' AUSENTE antes de salvar!")

            scores_path = output_path / "aurum_quality_scores_complete.parquet"
            self.scores_df.to_parquet(scores_path, index=False)

            scores_csv_path = output_path / "aurum_quality_scores_complete.csv"
            self.scores_df.to_csv(scores_csv_path, index=False, sep=';', float_format='%.4f', encoding='utf-8-sig')

            logger.info(f"💾 Scores completos (histórico) salvos em:")
            logger.info(f"   -> {scores_path}")
            logger.info(f"   -> {scores_csv_path}")

        latest_scores = self.get_latest_scores()
        
        latest_path = output_path / "aurum_quality_scores_latest.parquet"
        latest_csv_path = output_path / "aurum_quality_scores_latest.csv"
        
        latest_scores.to_parquet(latest_path, index=False)
        latest_scores.to_csv(latest_csv_path, index=False, sep=';', encoding='utf-8-sig')
        
        logger.info(f"💾 Scores mais recentes salvos: {latest_path} e {latest_csv_path}")

        # Salvar Estatísticas
        stats_path = output_path / "aurum_scores_statistics.txt"
        self._save_statistics(latest_scores, stats_path)
        logger.info(f"💾 Estatísticas salvas: {stats_path}")

        return {
            'complete_parquet': scores_path,
            'complete_csv': scores_csv_path,
            'latest_parquet': latest_path,
            'statistics': stats_path
        }

    def _save_statistics(self, scores_df: pd.DataFrame, stats_path: Path):
        with open(stats_path, 'w', encoding='utf-8') as f:
            f.write("AURUM QUALITY SCORE - ESTATÍSTICAS (ÚLTIMO PERÍODO)\n" + "=" * 50 + "\n\n")
            f.write(f"Total de empresas: {len(scores_df)}\n")
            max_date = scores_df['DT_FIM_EXERC'].max()
            f.write(f"Período mais recente: {max_date if pd.notna(max_date) else 'N/A'}\n\n")
            if 'aurum_quality_score' in scores_df.columns:
                 f.write("DISTRIBUIÇÃO DOS SCORES:\n" + f"  Média: {scores_df['aurum_quality_score'].mean():.2f}\n" +
                         f"  Mediana: {scores_df['aurum_quality_score'].median():.2f}\n" + f"  Mínimo: {scores_df['aurum_quality_score'].min():.2f}\n" +
                         f"  Máximo: {scores_df['aurum_quality_score'].max():.2f}\n\n")
            else: f.write("DISTRIBUIÇÃO DOS SCORES: N/A\n\n")
            if 'quality_grade' in scores_df.columns:
                 f.write("DISTRIBUIÇÃO POR NOTAS:\n")
                 grade_counts = scores_df['quality_grade'].value_counts().sort_index(ascending=True)
                 for grade, count in grade_counts.items(): f.write(f"  Nota {grade}: {count} empresas\n")
            else: f.write("DISTRIBUIÇÃO POR NOTAS: N/A\n")
            if 'aurum_quality_score' in scores_df.columns:
                 f.write("\nTOP 10 EMPRESAS:\n")
                 top_10 = scores_df.nlargest(10, 'aurum_quality_score')[['DENOM_CIA', 'aurum_quality_score', 'quality_grade']]
                 for i, (_, row) in enumerate(top_10.iterrows(), 1):
                      grade = row.get('quality_grade', 'N/A')
                      f.write(f"  {i:2d}. {str(row['DENOM_CIA'])[:35]:35} {row['aurum_quality_score']:6.2f} (Nota {grade})\n")
            else: f.write("\nTOP 10 EMPRESAS: N/A\n")


# ==================== EXECUÇÃO PRINCIPAL ====================
def main():
    logger.info("🚀 INICIANDO PIPELINE DO AURUM QUALITY SCORE (Versão Final com ROIC Correto)")
    try:
        calculator = AurumQualityScoreCalculator(
            fundamentals_path="../data/cvm/final/fundamentals_wide.parquet"
        )
        calculator.load_and_prepare_data()
        calculator.calculate_ttm_data()
        calculator.calculate_financial_ratios() # Calcular ROIC correto e ratios de dívida
        calculator.calculate_quality_scores()   # Usar ROIC correto e ratios de dívida
        output_files = calculator.save_results(output_dir="../data/aurum_scores") # Salvar no diretório correto
        logger.info(f"💾 Resultados (com ROIC correto) salvos em: {output_files}")

        latest_scores = calculator.get_latest_scores()
        # ... (impressão dos resultados igual) ...
        print("\n" + "="*60 + "\n🎉 AURUM QUALITY SCORE - RESULTADOS FINAIS (ÚLTIMO PERÍODO)\n" + "="*60)
        print(f"\n📊 TOTAL DE EMPRESAS: {len(latest_scores)}")
        if 'aurum_quality_score' in latest_scores.columns: print(f"📈 SCORE MÉDIO: {latest_scores['aurum_quality_score'].mean():.2f}")
        print(f"\n📋 DISTRIBUIÇÃO DAS NOTAS:")
        if 'quality_grade' in latest_scores.columns:
             grade_dist = latest_scores['quality_grade'].value_counts().sort_index(ascending=True)
             for grade, count in grade_dist.items(): print(f"   Nota {grade}: {count:3d} empresas")
        print(f"\n🥇 TOP 10 EMPRESAS:")
        if 'aurum_quality_score' in latest_scores.columns:
             top_10 = latest_scores.nlargest(10, 'aurum_quality_score')[['DENOM_CIA', 'aurum_quality_score', 'quality_grade']]
             for i, (_, row) in enumerate(top_10.iterrows(), 1):
                  grade = row.get('quality_grade', 'N/A')
                  print(f"   {i:2d}. {str(row['DENOM_CIA'])[:35]:35} {row['aurum_quality_score']:6.2f} (Nota {grade})")

        return calculator

    except Exception as e:
        logger.error(f"❌ ERRO NO PIPELINE: {e}")
        import traceback
        logger.error(traceback.format_exc())
        raise

if __name__ == "__main__":
    aurum_calculator = main()

2025-12-17 17:50:20,381 - INFO - 🚀 INICIANDO PIPELINE DO AURUM QUALITY SCORE (Versão Final com ROIC Correto)
2025-12-17 17:50:20,400 - INFO - 📥 1. CARREGANDO NOVO fundamentals_wide.parquet...
2025-12-17 17:50:20,730 - INFO - ✅ Dados carregados: (21559, 19)
2025-12-17 17:50:20,732 - INFO - Colunas encontradas: ['CNPJ_CIA', 'DENOM_CIA', 'DT_FIM_EXERC', 'Custo dos Bens e/ou Serviços Vendidos', 'EBIT', 'EBT', 'Lucro Bruto', 'Lucro Líquido Consolidado', 'Receita Líquida', 'Ativo Circulante', 'Ativo Não Circulante', 'Ativo Total', 'Caixa e Equivalentes', 'Dívida Curto Prazo', 'Dívida Longo Prazo', 'Passivo Circulante', 'Passivo Não Circulante', 'Passivo Total', 'Patrimônio Líquido Consolidado']
2025-12-17 17:50:20,740 - INFO - ✅ Novas colunas (Caixa, Dívida CP, Dívida LP) encontradas!
2025-12-17 17:50:20,861 - INFO - ⏳ 2. CALCULANDO TTM...
100%|██████████| 729/729 [00:03<00:00, 196.08it/s]
2025-12-17 17:50:24,651 - INFO - ✅ TTM calculado.
2025-12-17 17:50:24,653 - INFO - 🧮 3. CALCULANDO RATI


🎉 AURUM QUALITY SCORE - RESULTADOS FINAIS (ÚLTIMO PERÍODO)

📊 TOTAL DE EMPRESAS: 729
📈 SCORE MÉDIO: 49.27

📋 DISTRIBUIÇÃO DAS NOTAS:
   Nota A: 191 empresas
   Nota B:  90 empresas
   Nota C:  83 empresas
   Nota D: 147 empresas
   Nota E: 218 empresas

🥇 TOP 10 EMPRESAS:
    1. CAMIL ALIMENTOS S.A.                100.00 (Nota A)
    2. MINUPAR PARTICIPACOES S.A.           90.07 (Nota A)
    3. 521 PARTICIPACOES S.A. - EM LIQUIDA  89.17 (Nota A)
    4. STEIN SP II PARTICIPAÇÕES S.A.       87.37 (Nota A)
    5. COMERCIAL QUINTELLA COM EXP SA EM L  84.67 (Nota A)
    6. SONDOTECNICA ENGENHARIA SOLOS S.A.   83.36 (Nota A)
    7. EPR INFRAESTRUTURA PR S.A.           83.19 (Nota A)
    8. MÉLIUZ S.A.                          82.29 (Nota A)
    9. RIVA INCORPORADORA S.A               79.77 (Nota A)
   10. AURA ALMAS MINERAÇÃO S.A.            79.68 (Nota A)



### 🧬 Script: Aurum Data Unifier (Merge Engine)

> **Fase:** Engenharia de Dados / ETL
> **Arquivo Alvo:** `aurum_master_features.parquet`

Este script é o **Motor de Integração** do sistema Aurum. Ele é responsável por fundir três fontes de dados heterogêneas (Preços, Fundamentos e Metadados) em um único DataFrame temporalmente coerente ("Master Features").

Sua principal função é resolver o problema do **"Look-Ahead Bias"** (viés de olhar para o futuro), aplicando um `LAG` (atraso) de 3 meses nas datas de balanço para simular a disponibilidade real da informação para o investidor.

### 🎯 Objetivos

1. **Sincronização Temporal:** Alinhar preços diários/mensais com fundamentos trimestrais.
2. **Mapeamento de Identificadores:** Conectar `CNPJ` (usado nos balanços da CVM) com `Tickers` (usados na bolsa).
3. **Cálculo de Volatilidade:** Gerar a métrica de risco (`StdDev` de 63 dias) para uso no scoring.
4. **Consolidação:** Gerar o arquivo final que alimentará o Backtest e o Scoring Engine.

### 🏗️ Arquitetura de Dados (Data Flow)

O processo segue um fluxo linear de carregamento, transformação e fusão (`Merge As-Of`).

```mermaid
flowchart TD
    %% --- FONTES ---
    subgraph Sources ["1. Fontes de Dados"]
        Prices[("<b>Preços Limpos</b><br/>all_histories_cleaned.parquet")]
        Fund[("<b>Scores Fundamentais</b><br/>aurum_quality_scores_complete.parquet")]
        Map[("<b>Mapa Ticker-CNPJ</b><br/>mapa_ticker_cnpj.parquet")]
    end

    %% --- VOLATILIDADE ---
    subgraph Volatility ["2. Cálculo de Risco"]
        CalcVol["<b>Cálculo de Volatilidade</b><br/>Janela Móvel 63 dias"]
        Resample["<b>Agregação Mensal</b><br/>Resample('M').last()"]
    end

    %% --- PREPARAÇÃO ---
    subgraph Prep ["3. Preparação & Lag"]
        Melt["<b>Melt Preços</b><br/>Wide -> Long Format"]
        Lag["<b>Aplicação de Lag (3 Meses)</b><br/>Date_Balanço + 3M = Date_Disponível"]
        MapJoin["<b>Join CNPJ-Ticker</b><br/>Vínculo da empresa com código de bolsa"]
    end

    %% --- FUSÃO ---
    subgraph Merge ["4. Fusão Temporal (As-Of)"]
        AsOf{{"<b>Merge As-Of</b>"}}
        Note["Alinha o preço de HOJE<br/>com o balanço MAIS RECENTE disponível"]
        VolJoin["<b>Join Volatilidade</b><br/>Left Join por Ticker/Data"]
    end

    %% --- OUTPUT ---
    subgraph Output ["5. Saída Final"]
        Clean["<b>Limpeza Final</b><br/>Remove linhas sem Score/Preço"]
        Master[("<b>AURUM MASTER FEATURES</b><br/>Dataset Unificado")]
    end

    %% --- LIGAÇÕES ---
    Prices --> CalcVol --> Resample
    Prices --> Melt --> AsOf
    Fund --> Lag --> MapJoin
    Map --> MapJoin --> AsOf
    Resample --> VolJoin
    
    AsOf --> VolJoin --> Clean --> Master

```

### ⚙️ Detalhes da Implementação

#### 1. Cálculo de Volatilidade (`calcular_volatilidade_mensal`)

* **Input:** Preços diários limpos.
* **Lógica:**
* Calcula retorno diário (`pct_change`).
* Calcula desvio padrão móvel de 63 dias (aprox. 1 trimestre útil).
* Agrega o último valor de cada mês (`resample('M').last()`).


* **Por que?** A volatilidade é um componente chave do Score Aurum (peso 10%), penalizando ativos instáveis.

#### 2. Tratamento de Fundamentos & Lag

* **Input:** `aurum_quality_scores_complete.parquet` (Dados da CVM).
* **O Problema:** Um balanço do 4º Trimestre (31/Dez) só é divulgado em Março/Abril. Se usarmos a data 31/Dez no backtest, estamos "roubando", pois a informação não existia publicamente naquele dia.
* **A Solução (Lag):**
```python
df_fund['date_disponivel'] = df_fund['date_balanco'] + DateOffset(months=3)

```


O sistema só permite que o robô "veja" o balanço 3 meses após o fechamento do trimestre.

#### 3. Mapeamento CNPJ -> Ticker

* A CVM usa `CNPJ` como chave primária. O Yahoo Finance usa `Ticker`.
* O script carrega um dicionário (`mapa_ticker_cnpj_automatizado.parquet`) para traduzir os dados.
* **Limpeza:** Remove caracteres especiais de CNPJ (`.`, `/`, `-`) para garantir o *match*.

#### 4. Merge As-Of (O Coração do Script)

* **Comando:** `pd.merge_asof(..., direction='backward')`
* **Como funciona:** Para cada data de preço (ex: 15/Jan/2024), ele procura a data de balanço disponível **mais próxima no passado** (ex: 30/Set/2023 + 3 meses lag = 30/Dez/2023).
* Isso garante que, em qualquer dia da simulação, temos o dado fundamentalista mais recente e válido.

### 📂 Outputs (Saídas)

O script gera dois arquivos idênticos em formatos diferentes:

1. **`../data/aurum_master_features.parquet`**:
* Arquivo binário otimizado para leitura rápida no Pandas/Backtrader.
* Contém: `date`, `ticker`, `Adj Close`, `aurum_quality_score`, `ROE`, `VOLATILIDADE`, etc.


2. **`../data/aurum_master_features.csv`**:
* Versão em texto para auditoria visual e debug rápido no Excel.



### 🚀 Como Executar

Este script depende da execução prévia dos scripts de **Coleta de Preços** e **Cálculo de Scores Fundamentais**.

```bash
python src/step_04_unificar_master.py

```

### 📋 Verificação de Sucesso

Se o script rodar corretamente, você verá no log:

```text
✅✅✅ ARQUIVO MESTRE GERADO: ../data/aurum_master_features.parquet ✅✅✅

```

E uma amostra final dos dados combinados:

```text
             date    ticker date_disponivel  Adj Close  aurum_quality_score      ROE
...  2024-12-31  WEGE3.SA      2024-09-30      35.40                65.40  0.2510

```

---

In [ ]:
import pandas as pd
import numpy as np
import os
import logging
from pathlib import Path
from pandas.tseries.offsets import DateOffset

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

PATH_PRECOS_LIMPOS = "../data/historical/all_histories_cleaned.parquet"
PATH_PRECOS_WIDE = "../data/historical/prices_close_wide.parquet"
PATH_FUNDAMENTOS = "../data/aurum_scores/aurum_quality_scores_complete.parquet"
PATH_DE_PARA = "../data/dados_mapeamento/mapa_ticker_cnpj_automatizado.parquet"
# SENTIMENTO: Desativado temporariamente
# PATH_SENTIMENTO = "../data/news/news_with_sentiment.parquet" 

OUTPUT_DIR = "../data"
OUTPUT_FILENAME = "aurum_master_features.parquet"
output_path = Path(OUTPUT_DIR) / OUTPUT_FILENAME

def calcular_volatilidade_mensal():
    logger.info("Iniciando Unificação (Passo 1/2): Cálculo da Volatilidade...")
    try: 
        df_prices_daily = pd.read_parquet(PATH_PRECOS_LIMPOS)
    except FileNotFoundError: 
        logger.error(f"Arquivo não encontrado: {PATH_PRECOS_LIMPOS}")
        return None
        
    df_prices_daily['date'] = pd.to_datetime(df_prices_daily['date'])
    df_prices_daily['returns'] = df_prices_daily.groupby('ticker')['Adj Close'].pct_change()
    
    df_prices_daily['VOLATILIDADE'] = df_prices_daily.groupby('ticker')['returns'].rolling(window=63, min_periods=30).std().reset_index(0, drop=True)
    
    df_vol_mensal = df_prices_daily.set_index('date').groupby('ticker').resample('M').last()['VOLATILIDADE'].reset_index()
    
    logger.info(f"✅ Volatilidade mensal calculada.")
    return df_vol_mensal

def carregar_e_limpar_mapeamento():
    """ Carrega e prepara o novo arquivo de mapeamento """
    logger.info(f"📂 Carregando mapeamento de: {PATH_DE_PARA}")
    try:
        df_map = pd.read_parquet(PATH_DE_PARA)
        
        cols_necessarias = ['ticker', 'CNPJ']
        if not all(col in df_map.columns for col in cols_necessarias):
            logger.error(f"Colunas {cols_necessarias} não encontradas no mapeamento. Colunas atuais: {df_map.columns}")
            return None
            
        df_map = df_map[cols_necessarias].copy()
        
        df_map = df_map.rename(columns={'CNPJ': 'CNPJ_CIA'})
        
        df_map['CNPJ_CIA'] = df_map['CNPJ_CIA'].astype(str).str.replace(r'[.\-/]', '', regex=True)
        
        df_map['ticker'] = df_map['ticker'].apply(lambda x: f"{x}.SA" if not str(x).endswith(".SA") else x)
        
        df_map = df_map.drop_duplicates(subset=['ticker'])
        
        logger.info(f"✅ Mapeamento carregado e limpo: {len(df_map)} tickers.")
        return df_map
        
    except FileNotFoundError:
        logger.error(f"Arquivo de mapeamento não encontrado: {PATH_DE_PARA}")
        return None

def unificar_dataframe_mestre(df_vol_mensal):
    logger.info("Iniciando Unificação (Passo 2/2): Junção do DataFrame Mestre (SEM SENTIMENTO)...")

    try:
        df_close_wide = pd.read_parquet(PATH_PRECOS_WIDE)
        df_base_mensal = df_close_wide.melt(ignore_index=False, var_name='ticker', value_name='Adj Close').reset_index()
        df_base_mensal = df_base_mensal.rename(columns={'index': 'date'})
        logger.info(f"Base de preços (wide) carregada.")
    except FileNotFoundError: 
        logger.error(f"Arquivo não encontrado: {PATH_PRECOS_WIDE}")
        return

    try:
        df_fund = pd.read_parquet(PATH_FUNDAMENTOS)
        df_fund = df_fund.rename(columns={'DT_FIM_EXERC': 'date_balanco'}) 
        df_fund['date_balanco'] = pd.to_datetime(df_fund['date_balanco'])
        
        logger.info("⏳ Aplicando LAG de 3 meses nas datas de balanço...")
        df_fund['date_disponivel'] = df_fund['date_balanco'] + DateOffset(months=3)
        
    except FileNotFoundError: 
        logger.error(f"Arquivo não encontrado: {PATH_FUNDAMENTOS}")
        return

    df_mapping = carregar_e_limpar_mapeamento()
    if df_mapping is None: return

    df_fund['CNPJ_CIA'] = df_fund['CNPJ_CIA'].astype(str).str.replace(r'[.\-/]', '', regex=True)
    
    df_fund_com_ticker = pd.merge(df_fund, df_mapping, on='CNPJ_CIA', how='inner')
    
    if df_fund_com_ticker.empty:
        logger.error("❌ O Merge entre Fundamentos e Tickers retornou vazio! Verifique os CNPJs.")
        return
        
    logger.info(f"Fundamentos mapeados: {len(df_fund_com_ticker)} registros.")

    df_master = df_base_mensal.sort_values(by='date')
    df_fund_com_ticker = df_fund_com_ticker.sort_values(by='date_disponivel') 

    df_master = pd.merge_asof(
        df_master, 
        df_fund_com_ticker, 
        left_on='date', 
        right_on='date_disponivel', 
        by='ticker', 
        direction='backward' 
    )
    
    logger.info("Merge 'as-of' com LAG concluído.")

    if df_vol_mensal is not None:
        df_master = pd.merge(df_master, df_vol_mensal, on=['date', 'ticker'], how='left')
        df_master['VOLATILIDADE'] = df_master.groupby('ticker')['VOLATILIDADE'].ffill()

    key_metrics = ['aurum_quality_score', 'ROE', 'ROIC'] 
    
    df_master_clean = df_master.dropna(subset=key_metrics)
    
    removed = len(df_master) - len(df_master_clean)
    logger.info(f"Limpeza final: {removed} linhas removidas (sem score fundamentalista).")

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    df_master_clean.to_parquet(output_path, index=False)
    logger.info(f"✅✅✅ ARQUIVO MESTRE GERADO: {output_path} ✅✅✅")

    output_csv_path = Path(OUTPUT_DIR) / "aurum_master_features.csv"
    df_master_clean.to_csv(output_csv_path, index=False, sep=';', float_format='%.4f', encoding='utf-8-sig')
    logger.info(f"✅✅✅ ARQUIVO MESTRE GERADO (CSV): {output_csv_path} ✅✅✅")

    cols_view = ['date', 'ticker', 'date_disponivel', 'Adj Close', 'aurum_quality_score', 'ROE']
    print("\n--- Amostra Final ---")
    try:
        display(df_master_clean[cols_view].tail(5))
    except NameError:
        print(df_master_clean[cols_view].tail(5).to_string())

    return df_master_clean

if __name__ == "__main__":
    df_vol = calcular_volatilidade_mensal()
    
    if df_vol is not None:
        unificar_dataframe_mestre(df_vol)

2025-12-17 17:44:09,894 - INFO - Iniciando Unificação (Passo 1/2): Cálculo da Volatilidade...
C:\Users\kaike\AppData\Local\Temp\ipykernel_22668\4189223198.py:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_vol_mensal = df_prices_daily.set_index('date').groupby('ticker').resample('M').last()['VOLATILIDADE'].reset_index()
C:\Users\kaike\AppData\Local\Temp\ipykernel_22668\4189223198.py:40: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_vol_mensal = df_prices_daily.set_index('date').groupby('ticker').resample('M').last()['VOLATILIDADE'].reset_index()
2025-12-17 17:44:11,734 - INFO - ✅ Volatilidade mensal calculada.
2025-12-17 17:44:11,738 - 


--- Amostra Final ---


,date,ticker,date_disponivel,Adj Close,aurum_quality_score,ROE
34914,2025-12-31,SANB11.SA,2025-12-30,32.029999,32.493761,0.000000
34915,2025-12-31,BRKM5.SA,2025-12-30,7.860000,40.404762,-0.021431
34917,2025-12-31,BRAV3.SA,2025-12-30,13.430000,50.768216,0.151608
34918,2025-12-31,CYRE3.SA,2025-12-30,32.740002,33.327166,0.004177
34919,2025-12-31,YDUQ3.SA,2012-06-30,12.360000,58.418262,0.076099



### 🧠 Script: Sentiment Fusion Engine (Merge V6)

> **Fase:** Engenharia de Dados / Enriquecimento
> **Arquivo Alvo:** `aurum_master_features_final.parquet`

Este script é responsável por **enriquecer** o dataset financeiro mestre com dados de Inteligência Artificial (Sentimento). Ele unifica múltiplas fontes de notícias (Google News e MarketAux), padroniza os scores de sentimento gerados pelo RoBERTa e os agrega em uma visão mensal para alinhar com a frequência dos balanços e preços.

### 🎯 Objetivos

1. **Unificação de Fontes:** Combinar feeds de notícias díspares (Google = Recente, MarketAux = Histórico) em um único fluxo de informação.
2. **Padronização:** Garantir que datas e scores sigam o mesmo formato, independentemente da origem.
3. **Agregação Temporal:** Transformar notícias diárias esparsas em um indicador mensal robusto (`SENTIMENT_SCORE` médio e `NEWS_VOLUME`).
4. **Integração Final:** Cruzar esses indicadores com o arquivo mestre financeiro (Preço + Fundamentos).

### 🏗️ Arquitetura de Dados (Data Flow)

O processo segue um fluxo de "Funil": coleta de várias pontas, agregação no meio e fusão final.

```mermaid
flowchart TD
    %% --- FONTES ---
    subgraph Sources ["1. Fontes de Dados"]
        Fin[("<b>Financeiro Mestre</b><br/>aurum_master_features.parquet")]
        GNews[("<b>Google News</b><br/>news_with_sentiment.parquet")]
        MktAux[("<b>MarketAux (Histórico)</b><br/>Pasta de Parquets")]
    end

    %% --- PREPARAÇÃO DE NOTÍCIAS ---
    subgraph NewsPrep ["2. Padronização de Notícias"]
        NormG["<b>Normalizar Google</b><br/>Rename Cols -> Ticker, Date, Score"]
        NormM["<b>Normalizar MarketAux</b><br/>Loop em Arquivos -> Concat"]
        Union["<b>União (Concat)</b><br/>Google + MarketAux"]
    end

    %% --- AGREGAÇÃO ---
    subgraph Agg ["3. Agregação Mensal"]
        GroupBy["<b>Agrupamento</b><br/>Por Ticker e Mês"]
        Metrics["<b>Cálculo de KPI</b><br/>Média (Sentimento)<br/>Contagem (Volume)"]
    end

    %% --- FUSÃO ---
    subgraph Merge ["4. Fusão Final"]
        LeftJoin{{"<b>Left Join</b>"}}
        Note["Preserva todas as linhas financeiras.<br/>Se não houver notícia, preenche com 0."]
        FillNA["<b>Tratamento de Nulos</b><br/>Sentimento = 0 (Neutro)"]
    end

    %% --- OUTPUT ---
    subgraph Output ["5. Saída"]
        Final[("<b>DATASET FINAL V6</b><br/>aurum_master_features_final.parquet")]
    end

    %% --- LIGAÇÕES ---
    GNews --> NormG
    MktAux --> NormM
    NormG & NormM --> Union
    Union --> GroupBy --> Metrics
    
    Fin --> LeftJoin
    Metrics --> LeftJoin
    LeftJoin --> FillNA --> Final

```

### ⚙️ Detalhes da Implementação

#### 1. Carregamento Flexível

* **Google News:** Arquivo único, geralmente contendo dados recentes.
* **MarketAux:** Diretório com múltiplos arquivos `.parquet` particionados. O script itera sobre todos eles (`glob`), garantindo que nenhum dado histórico seja perdido.

#### 2. Agregação Mensal (O "Fator Sentimento")

Como o preço e os fundamentos já estão alinhados mensalmente, não podemos usar notícias diárias diretamente (daria erro de granularidade).
O script cria uma chave `periodo` (Ano-Mês) e calcula:

* **`SENTIMENT_SCORE`:** A média de todos os scores de notícias daquele mês.
* **`NEWS_VOLUME`:** Quantas notícias saíram sobre a empresa.
* **`SENTIMENT_VOL`:** (Opcional) O desvio padrão, para medir se as opiniões estão divididas.

#### 3. Tratamento de Ausência (Neutralidade)

Nem toda empresa tem notícia todo mês.

* **Lógica:** `fillna(0.0)`
* **Interpretação:** "No News is Good News" (ou pelo menos, não é bad news). A ausência de notícias é tratada como **Sentimento Neutro**, garantindo que o algoritmo de Scoring não quebre ao encontrar um valor nulo.

#### 📂 Outputs (Saídas)

O arquivo gerado (`../data/aurum_master_features_final.parquet`) é o **Artefato Final** da engenharia de dados. Ele contém:

| Coluna | Descrição |
| --- | --- |
| `date` | Data de referência (fim do mês). |
| `ticker` | Código do ativo (ex: PETR4.SA). |
| `Adj Close` | Preço ajustado. |
| `ROE`, `Dívida...` | Indicadores fundamentalistas (com Lag). |
| **`SENTIMENT_SCORE`** | **Novo:** Nota de qualidade das notícias (-1 a 1). |
| **`NEWS_VOLUME`** | **Novo:** Quantidade de notícias no mês. |

#### 🚀 Como Executar

Este script deve ser rodado **após** a unificação financeira (Step 04) e o processamento de NLP.

```bash
python src/step_06_merge_final_sentiment.py

```

#### 📋 Verificação de Sucesso

No final da execução, o script exibe uma amostra dos meses com maior volume de notícias ("Buzz"). Verifique se os dados fazem sentido:

```text
🔎 Top 5 Meses com Mais Notícias:
            date    ticker  SENTIMENT_SCORE  NEWS_VOLUME
 2023-01-31  AMER3.SA        -0.8500        150.0   <-- Ex: Crise Americanas
 2021-02-28  PETR4.SA        -0.4200         85.0

```

---

In [2]:
import pandas as pd
import glob
import os
import logging
from pathlib import Path

# --- CONFIGURAÇÃO DE CAMINHOS (AJUSTADO CONFORME SEU PEDIDO) ---
# 1. Dados Financeiros (Preço + Fundamentos)
PATH_MESTRE_ANTIGO = "../data/aurum_master_features.parquet"

# 2. Fonte Google News (Arquivo Único)
PATH_GOOGLE_NEWS = "../data/news/news_with_sentiment.parquet" 

# 3. Fonte MarketAux (Pasta com Parquets processados pelo RoBERTa)
DIR_MARKETAUX = "../data/news/processed/marketaux_roberta"

# 4. Saída Final
OUTPUT_FILE = "../data/aurum_master_features_final.parquet"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

def carregar_google():
    """Carrega e padroniza o Google News"""
    if not os.path.exists(PATH_GOOGLE_NEWS):
        logging.warning(f"⚠️ Google News não encontrado em: {PATH_GOOGLE_NEWS}")
        return pd.DataFrame()
        
    try:
        df = pd.read_parquet(PATH_GOOGLE_NEWS)
        # Padroniza colunas
        # Seu arquivo Google tem: ticker_query, title, sentiment_weighted (ou numeric_sentiment), published_date
        cols_map = {
            'ticker_query': 'ticker',
            'published_date': 'date',
            'sentiment_weighted': 'score'
        }
        # Fallback se não tiver o weighted
        if 'sentiment_weighted' not in df.columns and 'numeric_sentiment' in df.columns:
            cols_map['numeric_sentiment'] = 'score'
            
        df = df.rename(columns=cols_map)
        df['source'] = 'Google'
        
        # Garante data limpa
        df['date'] = pd.to_datetime(df['date'], errors='coerce', utc=True).dt.tz_localize(None)
        
        # Filtra colunas vitais
        cols = ['date', 'ticker', 'score', 'title', 'source']
        return df[[c for c in cols if c in df.columns]]
        
    except Exception as e:
        logging.error(f"Erro ao ler Google News: {e}")
        return pd.DataFrame()

def carregar_marketaux():
    """Carrega e padroniza o MarketAux (Múltiplos arquivos)"""
    if not os.path.exists(DIR_MARKETAUX):
        logging.warning(f"⚠️ Pasta MarketAux não encontrada: {DIR_MARKETAUX}")
        return pd.DataFrame()
        
    arquivos = list(Path(DIR_MARKETAUX).glob("*.parquet"))
    logging.info(f"📂 Lendo {len(arquivos)} arquivos do MarketAux...")
    
    dfs = []
    for f in arquivos:
        try:
            df = pd.read_parquet(f)
            # Padroniza
            if 'sentiment' in df.columns:
                df = df.rename(columns={'sentiment': 'score'})
            
            df['source'] = 'MarketAux'
            df['date'] = pd.to_datetime(df['date'], errors='coerce', utc=True).dt.tz_localize(None)
            
            cols = ['date', 'ticker', 'score', 'title', 'source']
            # Garante que existem (Title pode não existir em alguns processados antigos)
            cols_validas = [c for c in cols if c in df.columns]
            dfs.append(df[cols_validas])
        except: pass
        
    if not dfs: return pd.DataFrame()
    return pd.concat(dfs, ignore_index=True)

def main():
    print("--- 🦁 INICIANDO MERGE FINAL (V6 - FONTES CORRETAS) ---")
    
    # 1. Carrega as duas fontes
    df_google = carregar_google()
    df_mkt = carregar_marketaux()
    
    logging.info(f"📊 Linhas -> Google: {len(df_google)} | MarketAux: {len(df_mkt)}")
    
    if df_google.empty and df_mkt.empty:
        logging.error("❌ Nenhuma notícia carregada. Abortando.")
        return

    # 2. Unifica
    df_news = pd.concat([df_google, df_mkt], ignore_index=True)
    
    # 3. Agregação Mensal (Média do Score no Mês)
    logging.info("🔄 Calculando 'Fator Sentimento' Mensal...")
    
    # Cria chave Mês
    df_news['periodo'] = df_news['date'].dt.to_period('M')
    
    df_agregado = df_news.groupby(['ticker', 'periodo']).agg(
        SENTIMENT_SCORE=('score', 'mean'),      # Qualidade (Positivo/Negativo)
        NEWS_VOLUME=('score', 'count'),         # Quantidade (Buzz)
        SENTIMENT_VOL=('score', 'std')          # Volatilidade de opinião (Polêmica)
    ).reset_index()
    
    logging.info(f"✅ Base Agregada: {len(df_agregado)} linhas (Ticker x Mês).")

    # 4. Cruzamento com Financeiro
    if not os.path.exists(PATH_MESTRE_ANTIGO):
        logging.error("❌ Arquivo Mestre Financeiro não encontrado.")
        return

    df_mestre = pd.read_parquet(PATH_MESTRE_ANTIGO)
    df_mestre['date'] = pd.to_datetime(df_mestre['date'])
    df_mestre['periodo'] = df_mestre['date'].dt.to_period('M')
    
    logging.info("🔗 Realizando Merge (Left Join)...")
    
    df_final = pd.merge(
        df_mestre,
        df_agregado[['ticker', 'periodo', 'SENTIMENT_SCORE', 'NEWS_VOLUME']],
        on=['ticker', 'periodo'],
        how='left'
    )
    
    # 5. Tratamento de Nulos
    # Se não teve notícia, Sentimento = 0 (Neutro)
    df_final['SENTIMENT_SCORE'] = df_final['SENTIMENT_SCORE'].fillna(0.0)
    df_final['NEWS_VOLUME'] = df_final['NEWS_VOLUME'].fillna(0.0)
    
    # Limpeza
    df_final = df_final.drop(columns=['periodo'])
    
    # 6. Salva
    df_final.to_parquet(OUTPUT_FILE, index=False)
    
    print("\n" + "="*40)
    print(f"🏆 DATASET V4 PRONTO: {OUTPUT_FILE}")
    print("="*40)
    
    # Amostra dos dados com notícias
    print("\n🔎 Top 5 Meses com Mais Notícias:")
    print(df_final.sort_values('NEWS_VOLUME', ascending=False)[['date', 'ticker', 'SENTIMENT_SCORE', 'NEWS_VOLUME']].head(5).to_string())

if __name__ == "__main__":
    main()

--- 🦁 INICIANDO MERGE FINAL (V6 - FONTES CORRETAS) ---


2025-12-20 02:40:15,747 - 📂 Lendo 97 arquivos do MarketAux...
2025-12-20 02:40:16,583 - 📊 Linhas -> Google: 2766 | MarketAux: 204
2025-12-20 02:40:16,588 - 🔄 Calculando 'Fator Sentimento' Mensal...
2025-12-20 02:40:16,614 - ✅ Base Agregada: 287 linhas (Ticker x Mês).
2025-12-20 02:40:16,650 - 🔗 Realizando Merge (Left Join)...



🏆 DATASET V4 PRONTO: ../data/aurum_master_features_final.parquet

🔎 Top 5 Meses com Mais Notícias:
            date    ticker  SENTIMENT_SCORE  NEWS_VOLUME
12561 2022-05-31  PETR4.SA        -0.111604         15.0
12486 2022-05-01  PETR4.SA        -0.111604         15.0
12982 2022-08-31  BBAS3.SA        -0.281047          9.0
12299 2022-03-31  PETR4.SA         0.105945          9.0
12219 2022-03-01  PETR4.SA         0.105945          9.0




### 🧮 Script: Aurum Scoring Engine (Final)

> **Fase:** Inteligência / Modelagem
> **Arquivo Alvo:** `aurum_scored_history.parquet`

Este script é o **Coração Quantitativo** do sistema Aurum. Ele aplica um modelo multifatorial para transformar métricas financeiras brutas (que possuem escalas diferentes, como `%` e `R$`) em uma nota padronizada de 0 a 100, integrando também os dados de Sentimento (IA) e Risco (Volatilidade).

### 🎯 Objetivos

1. **Normalização Estatística:** Tornar comparáveis métricas de naturezas distintas (ex: comparar ROE com Sentimento).
2. **Ponderação de Fatores:** Aplicar a "Tese de Investimento" atribuindo pesos específicos para Rentabilidade, Solvência e Risco.
3. **Classificação:** Gerar o **Aurum Quality Score** e atribuir notas (Grades A-E) para facilitar a seleção de ativos.

### 🏗️ Arquitetura Lógica (Algoritmo)

O motor utiliza uma função **Sigmoide sobre Z-Score** para suavizar outliers e garantir uma distribuição justa de notas.

```mermaid
flowchart TD
    %% --- INPUT ---
    Input[("<b>Input: Master Features</b><br/>aurum_master_features_final.parquet")]

    %% --- CONFIGURAÇÃO ---
    subgraph Config ["1. Definição de Pesos (Tese)"]
        Rent["<b>Rentabilidade (45%)</b><br/>ROE, ROA, Margens"]
        Solv["<b>Solvência (30%)</b><br/>Dívida/PL, Liquidez"]
        Risk["<b>Risco & IA (25%)</b><br/>Sentimento (NLP) + Volatilidade"]
    end

    %% --- NORMALIZAÇÃO (LOOP) ---
    subgraph Norm ["2. Pipeline de Normalização (Por Métrica)"]
        Winsor["<b>Winsorization</b><br/>Clip Outliers (1% - 99%)"]
        ZScore["<b>Z-Score</b><br/>Desvio da Média"]
        Sigmoid["<b>Função Sigmoide</b><br/>Transformação para 0-1"]
        Scale["<b>Escala Final</b><br/>0 a 100"]
    end

    %% --- AGREGAÇÃO ---
    subgraph Agg ["3. Cálculo Final"]
        Weighted["<b>Média Ponderada</b><br/>Soma (Nota * Peso)"]
        Grade["<b>Classificação (Grade)</b><br/>A (>80), B (>60)..."]
    end

    %% --- OUTPUTS ---
    subgraph Output ["4. Saída"]
        Hist[("<b>Histórico Completo</b><br/>aurum_scored_history.parquet")]
        Snapshot[("<b>Carteira Atual (Snapshot)</b><br/>aurum_scored_latest_portfolio.csv")]
    end

    %% --- LIGAÇÕES ---
    Input --> Rent & Solv & Risk
    Rent & Solv & Risk --> Winsor
    Winsor --> ZScore --> Sigmoid --> Scale
    Scale --> Weighted --> Grade
    Grade --> Hist & Snapshot

```

### ⚙️ Detalhes da Implementação

#### 1. Métricas e Pesos

O sistema avalia 3 pilares fundamentais:

* **Rentabilidade (45%):** Eficiência em gerar lucro (`ROE`, `ROIC`, `Margens`).
* **Solvência (30%):** Saúde financeira (`Dívida/PL`, `Liquidez`).
* **Qualidade/Risco (25%):** O diferencial do Aurum (`Sentimento` positivo e `Volatilidade` baixa).

#### 2. O Segredo Matemático (Sigmoide)

Em vez de usar um ranking simples (que ignora a magnitude da diferença), usamos a Sigmoide:

* `Z = (Valor - Média) / DesvioPadrão`
* `Score = 1 / (1 + e^-Z)`
* **Por que?** Isso evita que uma única empresa com ROE de 500% distorça o ranking inteiro. A nota "satura" suavemente perto de 100.

#### 3. Tratamento de Direção

O script entende que para algumas métricas, **menos é mais**.

* **Exemplo:** Dívida Alta.
* O script inverte a nota automaticamente: `Score = 1 - Sigmoide`.

### 📂 Outputs (Saídas)

1. **`../data/aurum_final_scores/aurum_scored_history.parquet`**:
* O arquivo "Ouro". Contém todo o histórico de notas mês a mês. É este arquivo que alimenta o Backtest.


2. **`../data/aurum_final_scores/aurum_scored_latest_portfolio.csv`**:
* Um relatório executivo mostrando a situação **HOJE**.
* Ideal para abrir no Excel e decidir quais ações comprar na segunda-feira.



### 🚀 Como Executar

Este é o passo final antes do Backtest.

```bash
python src/step_08_scoring_engine_final.py

```

### 📋 Verificação de Sucesso

O script imprime o **Top 10 Atual** no console. Verifique se as empresas listadas são coerentes (Blue Chips de qualidade):

```text
🏆 TOP 10 AÇÕES HOJE (Qualidade Aurum):
 ticker  aurum_quality_score quality_grade       ROE  SENTIMENT_SCORE
 WEGE3.SA            78.45             B   0.2450             0.8
 PSSA3.SA            76.12             B   0.1800             0.5
 ...

```

---

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from typing import Dict, List, Optional, Tuple
import warnings
from dataclasses import dataclass
import json
import os

warnings.filterwarnings('ignore')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- CONFIGURAÇÃO ---
INPUT_FILE = "../data/aurum_master_features_final.parquet"
OUTPUT_DIR = "../data/aurum_final_scores"

@dataclass
class ScoringMetric:
    name: str
    weight: float
    direction: int 
    description: str
    min_value: float = None
    max_value: float = None
    ideal_range: Tuple[float, float] = None

class AurumScoringSystem:
    def __init__(self):
        self.scoring_metrics = self._initialize_scoring_metrics()
        self.quality_thresholds = {'A': 80, 'B': 60, 'C': 40, 'D': 20, 'E': 0}
        
    def _initialize_scoring_metrics(self) -> Dict[str, ScoringMetric]:
        metrics_config = {
            'ROE': ScoringMetric('ROE', 0.15, 1, 'Eficiência do capital próprio'),
            'ROA': ScoringMetric('ROA', 0.10, 1, 'Eficiência dos ativos'),
            'ROIC': ScoringMetric('ROIC', 0.10, 1, 'Retorno sobre capital investido'),
            'MARGEM_EBIT': ScoringMetric('MARGEM_EBIT', 0.05, 1, 'Margem Operacional'),
            'MARGEM_LIQUIDA': ScoringMetric('MARGEM_LIQUIDA', 0.05, 1, 'Margem Líquida'),

            'ALAVANCAGEM': ScoringMetric('ALAVANCAGEM', 0.08, -1, 'Passivo/Ativo (Menor é melhor)'),
            'DIVIDA_PL': ScoringMetric('DIVIDA_PL', 0.08, -1, 'Dívida/PL (Menor é melhor)'),
            'LIQUIDEZ_CORRENTE': ScoringMetric('LIQUIDEZ_CORRENTE', 0.07, 1, 'Liquidez de curto prazo'),
            'GIRO_ATIVO': ScoringMetric('GIRO_ATIVO', 0.07, 1, 'Eficiência de vendas'),

            'SENTIMENT_SCORE': ScoringMetric('SENTIMENT_SCORE', 0.15, 1, 'Sentimento de Notícias (IA)'),
            'VOLATILIDADE': ScoringMetric('VOLATILIDADE', 0.10, -1, 'Volatilidade do Preço (Menor é melhor)')
        }
        return metrics_config
    
    def calculate_individual_scores(self, df: pd.DataFrame) -> pd.DataFrame:
        scores_df = df.copy()
        
        cols_to_numeric = [m for m in self.scoring_metrics.keys() if m in scores_df.columns]
        for col in cols_to_numeric:
            scores_df[col] = pd.to_numeric(scores_df[col], errors='coerce')

        logger.info("🎯 Calculando scores individuais...")
        
        valid_metrics = []
        for metric_name, config in self.scoring_metrics.items():
            if metric_name not in scores_df.columns:
                logger.warning(f"⚠️ Métrica {metric_name} não encontrada. Peso será ignorado.")
                continue
            
            if scores_df[metric_name].abs().sum() == 0:
                 # logger.warning(f"⚠️ Métrica {metric_name} está vazia/zerada. Ignorando.")
                 pass

            series = scores_df[metric_name]
            lower = series.quantile(0.01)
            upper = series.quantile(0.99)
            series_clipped = series.clip(lower, upper)
            
            mean = series_clipped.mean()
            std = series_clipped.std()
            if std == 0: std = 1
            
            z_score = (series_clipped - mean) / std
            sigmoid_score = 1 / (1 + np.exp(-z_score)) 
            
            if config.direction == -1:
                sigmoid_score = 1 - sigmoid_score
                
            col_score = f'score_{metric_name}'
            scores_df[col_score] = sigmoid_score * 100
            
            valid_metrics.append((col_score, config.weight))
            
        return scores_df, valid_metrics
    
    def calculate_final_score(self, df: pd.DataFrame, metric_weights: List[Tuple]) -> pd.DataFrame:
        logger.info("⚖️ Calculando AURUM SCORE final...")
        
        total_weight = sum(w for _, w in metric_weights)
        if total_weight == 0: return df
        
        df['aurum_quality_score'] = 0.0
        
        for col_score, weight in metric_weights:
            adjusted_weight = weight / total_weight
            df['aurum_quality_score'] += df[col_score].fillna(50) * adjusted_weight
    
        conditions = [
            df['aurum_quality_score'] >= 80,
            df['aurum_quality_score'] >= 60,
            df['aurum_quality_score'] >= 40,
            df['aurum_quality_score'] >= 20
        ]
        choices = ['A', 'B', 'C', 'D']
        df['quality_grade'] = np.select(conditions, choices, default='E')
        
        return df

def run_scoring_pipeline():
    print("--- 🦁 INICIANDO MOTOR DE SCORING AVANÇADO ---")
    
    if not os.path.exists(INPUT_FILE):
        logger.error(f"Arquivo não encontrado: {INPUT_FILE}")
        return

    df = pd.read_parquet(INPUT_FILE)
    logger.info(f"Dados carregados: {len(df)} linhas de {len(df['ticker'].unique())} empresas.")
    
    engine = AurumScoringSystem()
    
    df_scored, valid_metrics = engine.calculate_individual_scores(df)
    
    df_final = engine.calculate_final_score(df_scored, valid_metrics)
    
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    
    file_hist = Path(OUTPUT_DIR) / "aurum_scored_history.parquet"
    df_final.to_parquet(file_hist, index=False)
    
    df_latest = df_final.sort_values('date').groupby('ticker').tail(1)
    file_latest = Path(OUTPUT_DIR) / "aurum_scored_latest_portfolio.csv"
    df_latest.to_csv(file_latest, sep=';', decimal=',', index=False)
    
    print("\n" + "="*50)
    print("✅ SCORING CONCLUÍDO!")
    print(f"📁 Histórico: {file_hist}")
    print(f"📁 Snapshot (Excel): {file_latest}")
    print("="*50)
    
    print("\n🏆 TOP 10 AÇÕES HOJE (Qualidade Aurum):")
    cols_show = ['ticker', 'aurum_quality_score', 'quality_grade', 'ROE', 'SENTIMENT_SCORE', 'VOLATILIDADE']
    top10 = df_latest.sort_values('aurum_quality_score', ascending=False).head(10)
    print(top10[cols_show].to_string(index=False))

if __name__ == "__main__":
    run_scoring_pipeline()

2025-12-23 18:24:18,594 - INFO - Dados carregados: 18879 linhas de 94 empresas.
2025-12-23 18:24:18,608 - INFO - 🎯 Calculando scores individuais...


--- 🦁 INICIANDO MOTOR DE SCORING AVANÇADO ---


2025-12-23 18:24:18,969 - INFO - ⚖️ Calculando AURUM SCORE final...



✅ SCORING CONCLUÍDO!
📁 Histórico: ..\data\aurum_final_scores\aurum_scored_history.parquet
📁 Snapshot (Excel): ..\data\aurum_final_scores\aurum_scored_latest_portfolio.csv

🏆 TOP 10 AÇÕES HOJE (Qualidade Aurum):
   ticker  aurum_quality_score quality_grade      ROE  SENTIMENT_SCORE  VOLATILIDADE
 MULT3.SA            57.928712             C 2.201927              0.0      0.015106
 SLCE3.SA            57.340818             C 0.119173              0.0      0.012109
 CURY3.SA            57.197869             C 0.166912              0.0      0.015372
 MOTV3.SA            56.527477             C 0.110268              0.0      0.014928
 SUZB3.SA            56.399023             C 0.301152              0.0      0.012409
 ISAE4.SA            55.815113             C 0.158355              0.0      0.011797
 RAIL3.SA            55.372407             C 0.095788              0.0      0.022669
 EGIE3.SA            55.311749             C 0.158355              0.0      0.014555
 PCAR3.SA            53